In [52]:
import numpy
import matplotlib
%matplotlib inline
import scipy.special

In [53]:
class neuralNetwork:
    
    def __init__(self,inputnodes,hiddennodes, outputnodes, learningrate):
        self.inodes=inputnodes
        self.hnodes=hiddennodes
        self.onodes=outputnodes
        self.lr=learningrate
        self.whi=numpy.random.normal(0.0,pow(self.hnodes,-.5), (self.hnodes, self.inodes))
        self.who=numpy.random.normal(0.0,pow(self.onodes,-.5), (self.onodes, self.hnodes))
        self.activation_function= lambda x: scipy.special.expit(x)
        pass

    def train(self, inputs_list, targets_list):
        #converts input list to 2d array
        inputs=numpy.array(inputs_list, ndmin=2).T
        
        targets =numpy.array(targets_list,ndmin=2).T
        
        #calc signals input into hidden layer
        hidden_inputs=numpy.dot(self.whi, inputs)
        
        # activate hidden layer
        hidden_outputs=self.activation_function(hidden_inputs)
        
        
        #calc final outputs
        final_inputs =numpy.dot(self.who, hidden_outputs)
        
        # activate final output
        final_outputs=self.activation_function(final_inputs)
        
        #error
        output_errors=targets -final_outputs
        
        #hidden errors
        hidden_errors= numpy.dot(self.who.T, output_errors)
        
        #calc the who
        self.who += self.lr * numpy.dot((output_errors * final_outputs* (1.0- final_outputs)),numpy.transpose(hidden_outputs))
        
        self.whi += self.lr * numpy.dot((hidden_errors * hidden_outputs* (1.0- hidden_outputs)),numpy.transpose(inputs))
        
        pass
    
    def query(self, input_lists):
        #converts input list to 2d array
        inputs=numpy.array(input_lists, ndmin=2).T
        
        #calc signals input into hidden layer
        hidden_inputs=numpy.dot(self.whi, inputs)
        
        # activate hidden layer
        hidden_outputs=self.activation_function(hidden_inputs)
        
        
        #calc final outputs
        final_inputs =numpy.dot(self.who, hidden_outputs)
        
        # activate final output
        final_outputs=self.activation_function(final_inputs)
        
        return final_outputs
    
    


In [80]:
training_data_file=open("mnist_dataset/mnist_train_100.csv","r")
training_data_list=training_data_file.readlines()
training_data_file.close()

input_nodes=784
hidden_nodes=100
output_nodes=10
learning_rate=.22

n=neuralNetwork(input_nodes,hidden_nodes, output_nodes, learning_rate)
epochs=200
for e in range(epochs):
    for record in training_data_list:
        all_values=record.split(',')
        inputs =(numpy.asfarray(all_values[1:]) /255 * .99 ) + .01
        targets=numpy.zeros(output_nodes) + .01
        targets[int(all_values[0])]=.99
        n.train(inputs, targets)



input_nodes=3
hidden_nodes =3
output_nodes=3
learning_rate=0.3

n=neuralNetwork(input_nodes,hidden_nodes, output_nodes, learning_rate)
n.query([1.0,0.5,-1.5])

In [59]:
test_data_file =open("mnist_dataset/mnist_test_10.csv","r")
test_data_list=test_data_file.readlines()
test_data_file.close()

all_values=test_data_list[0].split(',')
print(all_values[0]),
image_array=numpy.asfarray(all_values[1:]).reshape(28,28)
matplotlib.pyplot.imshow(image_array)
matplotlib.pyplot.imshow(image_array, cmap="Greys")
matplotlib.pyplot.imshow(image_array, cmap="Greys",interpolation='None')

n.query(((numpy.asfarray(all_values[1:])) / 255.0 * .99)+ .01)

In [79]:
scorecard=[]
for record in test_data_list:
    all_values=record.split(',')
    correct_label=int(all_values[0])
    #print(correct_label,"correct label")
    inputs =(numpy.asfarray(all_values[1:])/255 *.99) + .01
    outputs=n.query(inputs)
    label=numpy.argmax(outputs)
    #print(label,"networks answer")
    if (label==correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
scorecard_array=numpy.array(scorecard)
print('training set size', len(training_data_list))
print('hidden layers',hidden_nodes)
print('learning rate', learning_rate)
print('epochs',epochs)
print("permformance =",scorecard_array.sum()/scorecard_array.size)

training set size 100
hidden layers 200
learning rate 0.22
epochs 200
permformance = 0.7


permformance = 0.7
